In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import rewriter

In [2]:
md = pd.read_csv('./tests/datasets/rounakbanik__movie-recommender-systems__movies_metadata.csv', low_memory=False)

# -- STEFANOS -- Replicate Data

In [3]:
factor = 20
md = pd.concat([md]*factor)

# -- STEFANOS -- Fuse `apply()`

*Original*: 1.32s

*Rewritten*: 1.1s

## Speedup: 1.2x

Here the speedup is small which makes sense because this is basically a cache-related transformation and you need to have a bunch of data to see the effect. I don't replicate data much here because this is a test not a benchmark and we don't want it to run for long.

In [4]:
%%time
%%rewrite
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []).head()

--------- Final Source -----------
def _REWR_fused_apply(_REWR_ser):
    if type(_REWR_ser) == pd.Series:
        _REWR_res = []
        _REWR_ls = _REWR_ser.tolist()
        for _REWR_it in _REWR_ls:
            _REWR_left = literal_eval(_REWR_it)
            _REWR_right = [i['name'] for i in _REWR_left] if isinstance(
                _REWR_left, list) else []
            _REWR_res.append(_REWR_right)
        return pd.Series(_REWR_res)
    else:
        return _REWR_ser.apply(literal_eval).apply(lambda x: [i['name'] for
            i in _REWR_left] if isinstance(_REWR_left, list) else [])
md['genres'] = _REWR_fused_apply(md['genres'].fillna('[]')).head()

CPU times: user 1.17 s, sys: 14.1 ms, total: 1.18 s
Wall time: 1.18 s
